Wavenet - language model - autoregressive model

In [57]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
words = [word.lower() for word in list(words)]

4945


In [91]:
# build vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
chars = [c for c in chars if c not in (' ', '-')]
# print(chars)

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [92]:
import random
random.seed(42)
random.shuffle(words)

In [101]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      if ch not in (' ', '-'):
          ix = stoi[ch]
          X.append(context)
          Y.append(ix)
          context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

In [102]:
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([28096, 8]) torch.Size([28096])
torch.Size([3491, 8]) torch.Size([3491])
torch.Size([3537, 8]) torch.Size([3537])


In [113]:
for x,y in zip(Xtr[:20], Ytr[:20]):
    # print(x)
    # print(y)
    # for ix in x:
        # print(ix)
    print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> r
.......r --> o
......ro --> s
.....ros --> a
....rosa --> l
...rosal --> i
..rosali --> a
.rosalia --> .
........ --> r
.......r --> o
......ro --> z
.....roz --> e
....roze --> l
...rozel --> l
..rozell --> e
.rozelle --> .
........ --> s
.......s --> t
......st --> a
.....sta --> c


In [116]:
# https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear

class Linear:
    def __init__(self, in_features, out_features, bias=True):
        self.weight = torch.randn((in_features, out_features)) / fan_in**0.5
        self.bias = torch.zeros(out_features) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

In [115]:
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

In [117]:
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []